In [7]:
from __future__ import division
from pandas import Series, DataFrame
from pandas import read_excel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tkFileDialog

In [10]:
bm = read_excel('bank-data.xlsx')

In [11]:
bm.head()

,age,sex,region,income,married,children,car,save_act,ch_act,mortgage,ira
0,48,FEMALE,INNER_CITY,17546.0,NO,1,NO,NO,NO,NO,YES
1,40,MALE,TOWN,30085.1,YES,3,YES,NO,YES,YES,NO
2,51,FEMALE,INNER_CITY,16575.4,YES,0,YES,YES,YES,NO,NO
3,23,FEMALE,TOWN,20375.4,YES,3,NO,NO,YES,NO,NO
4,57,FEMALE,RURAL,50576.3,YES,0,NO,YES,NO,NO,NO


In [12]:
bm.shape

(600, 11)

In [13]:
bm.dtypes

age           int64
sex          object
region       object
income      float64
married      object
children      int64
car          object
save_act     object
ch_act       object
mortgage     object
ira          object
dtype: object

In [15]:
from copy import deepcopy
bm1 = deepcopy(bm)

In [16]:
from sklearn.preprocessing import LabelEncoder

In [18]:
bm1["sex"].head()

0    FEMALE
1      MALE
2    FEMALE
3    FEMALE
4    FEMALE
Name: sex, dtype: object

In [20]:
bm1["sex"] = LabelEncoder().fit_transform(bm1["sex"])

In [21]:
bm1["sex"].head()

0    0
1    1
2    0
3    0
4    0
Name: sex, dtype: int64

In [22]:
for x in ["region", "married", "car", "save_act", "ch_act", "mortgage", "ira"]:
    bm1[x] = LabelEncoder().fit_transform(bm1[x])
bm1.head()

,age,sex,region,income,married,children,car,save_act,ch_act,mortgage,ira
0,48,0,0,17546.0,0,1,0,0,0,0,1
1,40,1,3,30085.1,1,3,1,0,1,1,0
2,51,0,0,16575.4,1,0,1,1,1,0,0
3,23,0,3,20375.4,1,3,0,0,1,0,0
4,57,0,1,50576.3,1,0,0,1,0,0,0


In [24]:
bm.select_dtypes(include=['object']).head()

,sex,region,married,car,save_act,ch_act,mortgage,ira
0,FEMALE,INNER_CITY,NO,NO,NO,NO,NO,YES
1,MALE,TOWN,YES,YES,NO,YES,YES,NO
2,FEMALE,INNER_CITY,YES,YES,YES,YES,NO,NO
3,FEMALE,TOWN,YES,NO,NO,YES,NO,NO
4,FEMALE,RURAL,YES,NO,YES,NO,NO,NO


In [26]:
bm.select_dtypes(include=['object']).columns

Index([u'sex', u'region', u'married', u'car', u'save_act', u'ch_act',
       u'mortgage', u'ira'],
      dtype='object')

In [27]:
list(bm.select_dtypes(include = ['object']).columns)

[u'sex',
 u'region',
 u'married',
 u'car',
 u'save_act',
 u'ch_act',
 u'mortgage',
 u'ira']

In [28]:
objcols = list(bm.select_dtypes(include = ['object']).columns)
for x in objcols:
    bm[x] = LabelEncoder().fit_transform(bm[x])

In [29]:
bm.dtypes

age           int64
sex           int64
region        int64
income      float64
married       int64
children      int64
car           int64
save_act      int64
ch_act        int64
mortgage      int64
ira           int64
dtype: object

In [30]:
bm.head()

,age,sex,region,income,married,children,car,save_act,ch_act,mortgage,ira
0,48,0,0,17546.0,0,1,0,0,0,0,1
1,40,1,3,30085.1,1,3,1,0,1,1,0
2,51,0,0,16575.4,1,0,1,1,1,0,0
3,23,0,3,20375.4,1,3,0,0,1,0,0
4,57,0,1,50576.3,1,0,0,1,0,0,0


In [38]:
X = bm.ix[:,0:-1].as_matrix()
y = bm.ira.as_matrix()

In [37]:
bm.ix[:,0:-1].as_matrix()

array([[ 48.,   0.,   0., ...,   0.,   0.,   0.],
       [ 40.,   1.,   3., ...,   0.,   1.,   1.],
       [ 51.,   0.,   0., ...,   1.,   1.,   0.],
       ..., 
       [ 31.,   0.,   3., ...,   1.,   0.,   0.],
       [ 29.,   1.,   0., ...,   1.,   0.,   1.],
       [ 38.,   1.,   3., ...,   0.,   1.,   1.]])

In [39]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.333)

In [40]:
X_train.shape

(400, 10)

In [47]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(criterion = 'gini', min_samples_split = 10).fit(X_train, y_train)

In [42]:
dtree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [48]:
dtree.tree_.node_count

87

In [49]:
y_pred = dtree.predict(X_test)

In [51]:
pd.crosstab(y_test, y_pred, rownames=['True'], colnames = ['Predicted'], margins = True)

Predicted,0,1,All
True,,,
0,103,8,111
1,19,70,89
All,122,78,200


In [52]:
from sklearn.metrics import classification_report, accuracy_score
print "Accuracy Score:", accuracy_score(y_test, y_pred), '\n\n', classification_report(y_test, y_pred)

Accuracy Score: 0.865 

             precision    recall  f1-score   support

          0       0.84      0.93      0.88       111
          1       0.90      0.79      0.84        89

avg / total       0.87      0.86      0.86       200



In [53]:
from sklearn.pipeline import Pipeline 
from sklearn.grid_search import GridSearchCV
pipeline = Pipeline([("dtree", DecisionTreeClassifier())])
# now the parameter space
parameters = { 'dtree__max_depth': (4, 5, 6, 7, 8, 9, 10),\
              'dtree__min_samples_split': (1, 2, 3),\
              'dtree__min_samples_leaf': (1, 2, 3) }
# now for grid search using 10 fold CV for each combination of parameters
# some scoring options:'accuracy', f1','mean_squared_error', 'precision','recall', 'roc_auc'
# n_jobs: number of cpus to use,  -1 means use all
import time
start_time = time.time() # so we can see how long things take
grid_search = GridSearchCV( pipeline, parameters, n_jobs =-1, verbose = 1,\
                           scoring ='accuracy', cv=10)
grid_search.fit( X_train, y_train) 
print 'Best score: %0.3f' % grid_search.best_score_ 
print 'Best parameters set:' 
best_parameters = grid_search.best_estimator_.get_params() 
for param_name in sorted( parameters.keys()): 
    print '\ t% s: %r' % (param_name, best_parameters[ param_name]) 
predictions = grid_search.predict( X_test) 
print "\n\n Accuracy Score:", accuracy_score(y_test, predictions), '\n\n', classification_report(y_test, predictions)
print "elaplsed time ", time.time() - start_time

Fitting 10 folds for each of 63 candidates, totalling 630 fits
Best score: 0.853
Best parameters set:
\ tdtree__max_depth: 6
\ tdtree__min_samples_leaf: 1
\ tdtree__min_samples_split: 3


 Accuracy Score: 0.895 

             precision    recall  f1-score   support

          0       0.87      0.95      0.91       111
          1       0.94      0.82      0.87        89

avg / total       0.90      0.90      0.89       200

elaplsed time  1.64126610756


[Parallel(n_jobs=-1)]: Done 630 out of 630 | elapsed:    1.5s finished
